## Complementary Notebook: Appropriate Operators to Approximate Connectives and Quantifiers

This notebook is a complement to the tutorial on operators.

Logical connectives in LTN are grounded using fuzzy semantics. However, while all fuzzy logic operators make sense when simply *querying* the language, not every operator is equally suited for *learning* in LTN. This notebook details the motivations behind some choice of operators over the others.

In [1]:
import logictensornetworks as ltn
import numpy as np
import torch

### Querying

One can access the most common fuzzy logic operators in `ltn.fuzzy_ops`. They are implemented using tensorflow primitives.

We here compare
- the product t-norm: $u \land_{\mathrm{prod}} v = uv$,
- the Lukasiewicz t-norm: $u \land_{\mathrm{luk}} v = \max(u+v-1,0)$,
- the minimum aggregator: $\min(u_1,\dots,u_n)$,
- the p-mean error aggregator (generalized mean of the deviations w.r.t. the truth): $\mathrm{pME}(u_1,\dots,u_n) = 1 - \biggl( \frac{1}{n} \sum\limits_{i=1}^n (1-u_i)^p \biggr)^{\frac{1}{p}}$.

Each operator obviously conveys very different meanings, but they can all make sense depending on the intent of the query.

In [2]:
x1 = torch.tensor(0.4)
x2 = torch.tensor(0.7)

# the stable keyword is explained at the end of the notebook
and_prod = ltn.fuzzy_ops.And_Prod(stable=False)
and_luk = ltn.fuzzy_ops.And_Luk()

print(and_prod(x1,x2))
print(and_luk(x1,x2))

tensor(0.2800)
tensor(0.1000)


In [3]:
xs = torch.tensor([1.,1.,1.,0.5,0.3,0.2,0.2,0.1])

# the stable keyword is explained at the end of the notebook
forall_min = ltn.fuzzy_ops.Aggreg_Min()
forall_pME = ltn.fuzzy_ops.Aggreg_pMeanError(p=4, stable=False)

print(forall_min(xs))
print(forall_pME(xs))

tensor(0.1000)
tensor(0.3134)


### Learning

While all operators are suitable in a querying setting, this not the case in a learning setting. Indeed, many fuzzy logic operators have derivatives not suitable for gradient-based algorithms. For more details, read [van Krieken et al., *Analyzing Differentiable Fuzzy Logic Operators*, 2020](https://arxiv.org/abs/2002.06100).

We here give simple illustrations of such gradient issues.

#### 1. Vanishing Gradients

Some operators have vanishing gradients on some part of their domains.

e.g. in $u \land_{\mathrm{luk}} v = \max(u+v-1,0)$, if $u+v-1 < 0$, the gradients vanish.


In [4]:
torch.max((x1+x2)-1,torch.tensor(0))

tensor(0.1000)

In [5]:
x1 = torch.tensor(0.3, requires_grad=True)
x2 = torch.tensor(0.5, requires_grad=True)

y = and_luk(x1,x2)
y.backward()
gradients = [x.grad for x in [x1,x2]]
print(y)
print(gradients)


tensor(0., grad_fn=<MaximumBackward>)
[tensor(0.), tensor(0.)]


#### 2. Single-Passing Gradients

Some operators have gradients propagating to only one input at a time, meaning that all other inputs will not benefit from learning at this step.

e.g. in $\min(u_1,\dots,u_n)$.

In [6]:
xs = torch.tensor([1.,1.,1.,0.5,0.3,0.2,0.2,0.1], requires_grad=True)

y = forall_min(xs, axis=0)
y.backward()
print(y)
print(xs.grad)

tensor(0.1000, grad_fn=<MinBackward0>)
tensor([0., 0., 0., 0., 0., 0., 0., 1.])


#### 3. Exploding Gradients

Some operators have exploding gradients on some part of their domains.

e.g. in $\mathrm{pME}(u_1,\dots,u_n) = 1 - \biggl( \frac{1}{n} \sum\limits_{i=1}^n (1-u_i)^p \biggr)^{\frac{1}{p}}$, on the edge case where all inputs are $1.0$.

In [7]:
xs = torch.tensor([1.,1.,1.], requires_grad=True)

y = forall_pME(xs,p=4)
y.backward()
print(y)
print(xs.grad)


tensor(1., grad_fn=<RsubBackward1>)
tensor([nan, nan, nan])


### Stable Product Configuration

#### Product Configuration

Our general recommendation is to use the following "product configuration":
* not: the standard negation  $\lnot u = 1-u$,
* and: the product t-norm $u \land v = uv$,
* or: the product t-conorm (probabilistic sum) $u \lor v = u+v-uv$,
* implication: the Reichenbach implication $u \rightarrow v = 1 - u + uv$,
* existential quantification ("exists"): the generalized mean (p-mean) $\mathrm{pM}(u_1,\dots,u_n) = \biggl( \frac{1}{n} \sum\limits_{i=1}^n u_i^p \biggr)^{\frac{1}{p}} \qquad p \geq 1$,
* universal quantification ("for all"): the generalized mean of "the deviations w.r.t. the truth" (p-mean error) $\mathrm{pME}(u_1,\dots,u_n) = 1 - \biggl( \frac{1}{n} \sum\limits_{i=1}^n (1-u_i)^p \biggr)^{\frac{1}{p}} \qquad p \geq 1$.

#### "Stable"

As is, this configuration is not fully exempt from issues. The product t-norm has vanishing gradients on the edge case $u=v=0$. The product t-conorm has vanishing gradients on the edge case $u=v=1$. The Reichenbach implication has vanishing gradients on the edge case $u=0$,$v=1$. p-mean has exploding gradients on the edge case $u_1=\dots=u_n=0$. p-mean error has exploding gradients on the edge case $u_1=\dots=u_n=1$.  
However, all these issues happen on edge cases and can easily be fixed using the following "trick":
- if the edge case happens when an input $u$ is $0$, we modify every input with $u' = (1-\epsilon)u+\epsilon$,
- if the edge case happens when an input $u$ is $1$, we modify every input with $u' = (1-\epsilon)u$,

where $\epsilon$ is a small positive value (e.g. $1\mathrm{e}{-5}$).

This gives us stabilized versions of such operators. One can trigger the stable behavior by using the boolean keyword `stable`. One can set a default value for `stable` when initializing the operator, or can use different values at each call of the operator.

In [8]:
xs = torch.tensor([1.,1.,1.], requires_grad=True)

y = forall_pME(xs,p=4,stable=True)
y.backward()
print(y)
print(xs.grad)


tensor(0.9999, grad_fn=<RsubBackward1>)
tensor([0.3333, 0.3333, 0.3333])


#### The hyper-parameter $p$ in the generalized means

$p$ offers flexibility in writing more or less strict formulas, to account for outliers in the data depending on the application. Note that this can have strong implications for training.

In [9]:
xs = torch.tensor([1.,1.,1.,0.5,0.3,0.2,0.2,0.1], requires_grad=True)

y = forall_pME(xs,p=4)
y.backward()
print(y)
print(xs.grad)

tensor(0.3134, grad_fn=<RsubBackward1>)
tensor([0.0000, 0.0000, 0.0000, 0.0483, 0.1325, 0.1977, 0.1977, 0.2815])


In [10]:
xs = torch.tensor([1.,1.,1.,0.5,0.3,0.2,0.2,0.1], requires_grad=True)

y = forall_pME(xs,p=20)
y.backward()
print(y)
print(xs.grad)

tensor(0.1816, grad_fn=<RsubBackward1>)
tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0734e-05, 6.4147e-03, 8.1100e-02,
        8.1100e-02, 7.6019e-01])


While it can be tempting to set a high value for $p$ when querying, in a learning setting, this can quickly lead to a "single-passing" operator that will focus too much on outliers at each step (i.e., gradients overfitting one input at this step, potentially harming the training of the others). We recommend not to set a too high $p$ when learning.
